In [6]:
import modulos as md
lista=['MKRUSDT']

lista=md.lista_de_monedas()

for symbol in lista:
    try:
        data = md.estrategia_santa(symbol,tp_flag = False)
        resultado = md.backtesting(data,plot_flag=False)
        if resultado['Profit Factor']>1:# and resultado[6]/resultado[17] >=0.5:
            print(f"{symbol} - retorno: {md.truncate(resultado[6],2)}% - trades: {resultado[17]} - Profit Factor: {resultado['Profit Factor']}")
    except Exception as falla:
        pass  



c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:313: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)
c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:313: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)


XVGUSDT - retorno: 0.05% - trades: 13 - Profit Factor: 2.379931722093687
